In [3]:
from ragraph.io.esl import from_esl
graph = from_esl(["./esl"])

from utils import set_edge_weights, annotate_variable_type
set_edge_weights(graph)
annotate_variable_type(graph)

In [5]:
import raesl.plot
from raesl.plot.view_funcs import multi_domain
from ragraph.analysis import cluster


nodes, edges = multi_domain(
    graph,
    depth=4
)

relations = [n for n in nodes if n.kind == "relation_spec"]
variables = [n for n in nodes if n.kind == "variable"]
components = [n for n in nodes if n.kind == "component"]

# Cluster components
cluster.tarjans_scc(
    graph=graph,
    leafs = relations+variables,
)

figure = raesl.plot.mdm(graph)
figure.write_image("../figures/mdm_clustered.svg")

from ragraph.analysis import sequence
import ragraph.plot

from raesl.plot.view_funcs import multi_domain

nodes, edges = multi_domain(
    graph,
    depth=4
)

_, _, sequenced_functions = sequence.utils.branchsort(algo=sequence.scc_tearing, graph=graph, algo_args=dict(inf=1, dep=1, mu=3), inplace=True, nodes=relations+variables, edge_weights=graph.weights)

# Reload figure based on sequence.
figure = ragraph.plot.mdm(
    leafs = components + sequenced_functions,
    edges = edges,
    node_kinds=["component", "variable", "relation_spec"],
    style= ragraph.plot.Style(
        piemap=dict(
            display="weights",
            mode= "relative",
            fields=["weight"]
        ),
    )
)

figure.write_image("../figures/mdm_clustered_sequenced.svg")
figure.show()